# Data Wrangling

## Spark Loading

In [1]:
%load_ext sparkmagic.magics

In [2]:
import os
from IPython import get_ipython
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-homework3", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [3]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-homework3 -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5353,application_1649870447656_1960,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [4]:
%%spark
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

## Stops 15km from Zürich HB (8503000) (lat, lon) = (47.378177, 8.540192)

In [11]:
%%spark
all_stops = spark.read.csv('/data/sbb/csv/allstops/stop_locations.csv')
all_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+----+----+
|    _c0|                 _c1|             _c2|             _c3| _c4| _c5|
+-------+--------------------+----------------+----------------+----+----+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|null|null|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|null|null|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|null|null|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|null|null|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|null|null|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|null|null|
|1100013|  Mambach, Silbersau|47.7395192233867|7.88223152899259|null|null|
|1100014|Fröhnd (Schwarzw)...|47.7543663509316|7.88913059037559|null|null|
|1100015|Fröhnd (Schwarzw)...|47.7605926689054|7.88553732923861|null|null|
|1100016|     Wembach (Baden)|47.7728317637339|7.88772023537933|null|null|
+-------+----------------

In [14]:
%%spark
all_stops = all_stops.withColumnRenamed('_c0','stop_id')
all_stops = all_stops.withColumnRenamed('_c1','stop_name')
all_stops = all_stops.withColumnRenamed('_c2','lat')
all_stops = all_stops.withColumnRenamed('_c3','lon')
all_stops = all_stops.drop('_c4','_c5')
all_stops.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----------------+----------------+
|stop_id|           stop_name|             lat|             lon|
+-------+--------------------+----------------+----------------+
|1100006|Zell (Wiesental),...|47.7046317515335|7.84777215468897|
|1100008|Zell (Wiesental),...|47.7100842702352|7.85964788274668|
|1100009|Zell (Wiesental),...|47.7131911044794|7.86290876722849|
|1100010|           Atzenbach|47.7146175266411| 7.8723500608659|
|1100011|     Mambach, Brücke|47.7282088873189| 7.8774704579861|
|1100012|  Mambach, Mühlschau|47.7340818684375| 7.8813871126254|
|1100013|  Mambach, Silbersau|47.7395192233867|7.88223152899259|
|1100014|Fröhnd (Schwarzw)...|47.7543663509316|7.88913059037559|
|1100015|Fröhnd (Schwarzw)...|47.7605926689054|7.88553732923861|
|1100016|     Wembach (Baden)|47.7728317637339|7.88772023537933|
+-------+--------------------+----------------+----------------+
only showing top 10 rows

In [15]:
%%spark
import geopy

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named geopy
Traceback (most recent call last):
ImportError: No module named geopy

